In [1]:
%pip install PyPDF2
import requests
import pandas as pd
import time
import os
import io
import re
import tempfile
from PyPDF2 import PdfReader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.9 MB/s eta 0:00:00


In [3]:

df = pd.read_csv('columns.csv')

df

,paper_id,methods,model1,model2,model3,model4,model5,model6
0,on-enhancing-speech-emotion-recognition-using,"GAN, Convolution",GAN,NaN,NaN,NaN,NaN,NaN
1,snap-ml-a-hierarchical-framework-for-machine,Logistic Regression,Logistic Regression,NaN,NaN,NaN,NaN,NaN
2,a-novel-hybrid-machine-learning-model-for,SVM,SVM,NaN,NaN,NaN,NaN,NaN
3,laplacian-smoothing-gradient-descent,Logistic Regression,Logistic Regression,NaN,NaN,NaN,NaN,NaN
4,examining-the-use-of-neural-networks-for,SVM,SVM,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1513,a-role-for-prior-knowledge-in-statistical,"Logistic Regression, SVM, Feature Selection",Logistic Regression,SVM,NaN,NaN,NaN,NaN
1514,using-machine-learning-to-calibrate-storm,Logistic Regression,Logistic Regression,NaN,NaN,NaN,NaN,NaN
1515,fit-a-fast-and-accurate-framework-for-solving,AutoEncoder,Autoencoder,NaN,NaN,NaN,NaN,NaN
1516,improving-protein-gamma-turn-prediction-using,Capsule Network,Capsule Network,NaN,NaN,NaN,NaN,NaN


In [6]:
!pip install pdfplumber
import pdfplumber
import requests
import re
import tempfile
import os
import pandas as pd
from tqdm import tqdm

import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)
# Load your DataFrame if not already
# df = pd.read_csv("your_input_file.csv")

df["discussion/findings"] = ""
df["section_type"] = ""

def get_pdf_url(paper_id):
    paper_endpoint = f"https://paperswithcode.com/api/v1/papers/{paper_id}/"
    response = requests.get(paper_endpoint)
    if response.status_code == 200:
        data = response.json()
        return data.get("url_pdf")
    return None

def download_pdf_from_url(pdf_url):
    try:
        response = requests.get(pdf_url, stream=True)
        if response.status_code == 200:
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
            with open(temp_file.name, "wb") as f:
                f.write(response.content)
            return temp_file.name
    except:
        return None
    return None

def extract_text_pdfplumber(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        return "\n".join([page.extract_text() or "" for page in pdf.pages])

# Define keywords and regex patterns
DISCUSSION_KEYWORDS = [
    "results", "discussion", "results and discussion", "discussion and results",
    "experimental results", "empirical results", "experiments", "evaluation",
    "analysis", "findings", "experimental evaluation", "empirical evaluation",
]

CONCLUSION_KEYWORDS = [
    "conclusion", "conclusions", "closing remarks", "final thoughts",
    "discussion and future work",
]

def build_patterns(keywords):
    patterns = []
    for kw in keywords:
        patterns.append(rf"(?i)\b(?:\d+|[IVX]+)?[\.\)]?\s*{re.escape(kw)}\b(.*?)(?=\n\s*(?:\d+|[IVX]+)?[\.\)]?\s+\w+|\n\s*references\b|\Z)")
    return patterns

DISCUSSION_PATTERNS = build_patterns(DISCUSSION_KEYWORDS)
CONCLUSION_PATTERNS = build_patterns(CONCLUSION_KEYWORDS)

def extract_section_with_fallback(text):
    for pattern in DISCUSSION_PATTERNS:
        m = re.search(pattern, text, re.DOTALL)
        if m and len(m.group(1).strip()) >= 100:
            print(f"🧪 Extracted {len(m.group(1).strip())} characters for DISCUSSION")
            return m.group(1).strip(), "discussion"


    for pattern in CONCLUSION_PATTERNS:
        m = re.search(pattern, text, re.DOTALL)
        if m and len(m.group(1).strip()) >= 100:
            print(f"📝 Extracted {len(m.group(1).strip())} characters for CONCLUSION")
            return m.group(1).strip(), "conclusion"

    return "Section not found or too short.", "none"

# Main loop with tqdm
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting sections"):
    paper_id = row.get("paper_id", row.get("id"))
    pdf_url = get_pdf_url(paper_id)

    if not pdf_url:
        continue

    pdf_file = download_pdf_from_url(pdf_url)
    if not pdf_file:
        continue

    try:
        full_text = extract_text_pdfplumber(pdf_file)
        extracted_text, section_type = extract_section_with_fallback(full_text)
        df.at[idx, "discussion/findings"] = extracted_text
        df.at[idx, "section_type"] = section_type
    except Exception as e:
        df.at[idx, "discussion/findings"] = "Error during extraction"
        df.at[idx, "section_type"] = "error"
    finally:
        os.remove(pdf_file)

# Save output
df.to_csv("papers_with_discussion.csv", index=False)
print("✅ Extraction complete. Results saved to papers_with_discussion.csv")



Extracting sections:   0%|          | 1/1518 [00:01<29:05,  1.15s/it]

🧪 Extracted 2925 characters for DISCUSSION


Extracting sections:   0%|          | 2/1518 [00:03<48:22,  1.91s/it]

📝 Extracted 2103 characters for CONCLUSION


Extracting sections:   0%|          | 3/1518 [00:05<47:27,  1.88s/it]

...

🧪 Extracted 1688 characters for DISCUSSION


Extracting sections: 100%|█████████▉| 1517/1518 [2:06:55<00:04,  4.83s/it]

🧪 Extracted 1369 characters for DISCUSSION


Extracting sections: 100%|██████████| 1518/1518 [2:06:57<00:00,  5.02s/it]

🧪 Extracted 1640 characters for DISCUSSION
✅ Extraction complete. Results saved to papers_with_discussion.csv


In [7]:
df

,paper_id,methods,model1,model2,model3,model4,model5,model6,discussion/findings,section_type
0,on-enhancing-speech-emotion-recognition-using,"GAN, Convolution",GAN,NaN,NaN,NaN,NaN,NaN,of a SVM trained on synthetic\nRealopenSMILE+i...,discussion
1,snap-ml-a-hierarchical-framework-for-machine,Logistic Regression,Logistic Regression,NaN,NaN,NaN,NaN,NaN,"InthisworkwehavedescribedSnapML,anewframeworkf...",conclusion
2,a-novel-hybrid-machine-learning-model-for,SVM,SVM,NaN,NaN,NaN,NaN,NaN,on the EyeNet. 47% : 53% 0.8973 0.8781\n61% : ...,discussion
3,laplacian-smoothing-gradient-descent,Logistic Regression,Logistic Regression,NaN,NaN,NaN,NaN,NaN,on both a quadratic function and a\nsimple fin...,discussion
4,examining-the-use-of-neural-networks-for,SVM,SVM,NaN,NaN,NaN,NaN,NaN,show tures used to classify the image datasets...,discussion
...,...,...,...,...,...,...,...,...,...,...
1513,a-role-for-prior-knowledge-in-statistical,"Logistic Regression, SVM, Feature Selection",Logistic Regression,SVM,NaN,NaN,NaN,NaN,over algorithmic feature selection\nmethods al...,discussion
1514,using-machine-learning-to-calibrate-storm,Logistic Regression,Logistic Regression,NaN,NaN,NaN,NaN,NaN,. Random forests (Breiman output focus on cohe...,discussion
1515,fit-a-fast-and-accurate-framework-for-solving,AutoEncoder,Autoencoder,NaN,NaN,NaN,NaN,NaN,could be\n𝑁 𝑀 NumberofMonteCarlosamples.\nobta...,discussion
1516,improving-protein-gamma-turn-prediction-using,Capsule Network,Capsule Network,NaN,NaN,NaN,NaN,NaN,were unsatisfactory with Matthew correlation c...,discussion


In [8]:
# Ensure 'discussion/findings' is a column and of type Series
if not isinstance(df['discussion/findings'], pd.Series):
    raise TypeError("Expected 'discussion/findings' to be a pandas Series column.")

# Helper function to determine if a section has real content
def has_content(text):
    if not isinstance(text, str):
        return False
    stripped = text.strip().lower()
    return bool(stripped and stripped not in ["section not found.", "section not found or too short.", "error during extraction"])

# Apply the function to the column
disc_mask = df['discussion/findings'].apply(has_content)

# Sanity check
assert disc_mask.dtype == bool, "disc_mask is not boolean"

# Split the DataFrame based on presence of content
df_with_section = df[disc_mask].copy()
df_without_section = df[~disc_mask].copy()

# Optional printouts
print(f"✅ Papers with usable discussion/conclusion: {len(df_with_section)}")
print(f"❌ Papers without usable section: {len(df_without_section)}")


✅ Papers with usable discussion/conclusion: 1476
❌ Papers without usable section: 42


In [9]:
import re

# Step 1: Save current filtered subset (optional)
df_with_section.to_csv("papers_with_section.csv", index=False)

# Step 2: Compute section length
df_with_section['section_length'] = df_with_section['discussion/findings'].apply(lambda x: len(str(x).strip()))

# Step 3: Clean common patterns
def clean_section(text):
    text = str(text)
    text = re.sub(r'References\s*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'(This paper is organized.*?)(\n|$)', '', text, flags=re.IGNORECASE)
    return text.strip()

df_with_section['section_cleaned'] = df_with_section['discussion/findings'].apply(clean_section)

# Step 4: Clip at 'References' if it's embedded
def clip_at_references(text):
    match = re.search(r'\bReferences\b', str(text), flags=re.IGNORECASE)
    return str(text)[:match.start()].strip() if match else str(text)

df_with_section['section_clipped'] = df_with_section['section_cleaned'].apply(clip_at_references)

# Step 5: Define flag thresholds
max_thresh = 15000
df_with_section['flag_too_short'] = df_with_section['section_length'] < 300
df_with_section['flag_too_long'] = df_with_section['section_length'] > max_thresh

# Step 6: Assign quality scores (you can tweak the logic)
def score_quality(length):
    score = 1
    if length >= 500:
        score += 1
    if length >= 1000:
        score += 1
    if length < max_thresh:
        score += 1
    return score

df_with_section['quality_score'] = df_with_section['section_length'].apply(score_quality)

# Step 7: Preview results
print(df_with_section[['section_length', 'flag_too_short', 'flag_too_long', 'quality_score']].head())

# Optional: Save cleaned and scored dataset
df_with_section.to_csv("papers_with_cleaned_sections.csv", index=False)


   section_length  flag_too_short  flag_too_long  quality_score
0            2925           False          False              4
1            2103           False          False              4
2             512           False          False              3
3            1854           False          False              4
4            1313           False          False              4


In [10]:
# Filter based on quality score and flag thresholds
final_df = df_with_section[
    (df_with_section['quality_score'] > 3) &
    (~df_with_section['flag_too_short']) &
    (~df_with_section['flag_too_long'])
].copy()

# Use the cleaned and clipped version as the final section text
final_df['section'] = final_df['section_clipped']

# Select final output columns
# Include model columns if they exist, else just paper_id + section
model_cols = [col for col in df_with_section.columns if col.startswith('model')]
cols_to_keep = ['paper_id'] + model_cols + ['section']

final_cleaned = final_df[cols_to_keep].reset_index(drop=True)

# Preview
print(final_cleaned.head())

# Optional: Save it
final_cleaned.to_csv("final_cleaned_sections.csv", index=False)

                                        paper_id               model1 model2  \
0  on-enhancing-speech-emotion-recognition-using                  GAN    NaN   
1   snap-ml-a-hierarchical-framework-for-machine  Logistic Regression    NaN   
2           laplacian-smoothing-gradient-descent  Logistic Regression    NaN   
3       examining-the-use-of-neural-networks-for                  SVM    NaN   
4    genesis-of-basic-and-multi-layer-echo-state          Autoencoder    NaN   

  model3 model4 model5 model6  \
0    NaN    NaN    NaN    NaN   
1    NaN    NaN    NaN    NaN   
2    NaN    NaN    NaN    NaN   
3    NaN    NaN    NaN    NaN   
4    NaN    NaN    NaN    NaN   

                                             section  
0  of a SVM trained on synthetic\nRealopenSMILE+i...  
1  InthisworkwehavedescribedSnapML,anewframeworkf...  
2  on both a quadratic function and a\nsimple fin...  
3  show tures used to classify the image datasets...  
4  are. Feature extraction-based methods such

In [11]:
final_cleaned

,paper_id,model1,model2,model3,model4,model5,model6,section
0,on-enhancing-speech-emotion-recognition-using,GAN,NaN,NaN,NaN,NaN,NaN,of a SVM trained on synthetic\nRealopenSMILE+i...
1,snap-ml-a-hierarchical-framework-for-machine,Logistic Regression,NaN,NaN,NaN,NaN,NaN,"InthisworkwehavedescribedSnapML,anewframeworkf..."
2,laplacian-smoothing-gradient-descent,Logistic Regression,NaN,NaN,NaN,NaN,NaN,on both a quadratic function and a\nsimple fin...
3,examining-the-use-of-neural-networks-for,SVM,NaN,NaN,NaN,NaN,NaN,show tures used to classify the image datasets...
4,genesis-of-basic-and-multi-layer-echo-state,Autoencoder,NaN,NaN,NaN,NaN,NaN,are. Feature extraction-based methods such as ...
...,...,...,...,...,...,...,...,...
683,a-generative-model-to-synthesize-eeg-data-for,GAN,SVM,DCGAN,NaN,NaN,NaN,show that the Epilepsy Society and Melbourne U...
684,using-machine-learning-to-calibrate-storm,Logistic Regression,NaN,NaN,NaN,NaN,NaN,. Random forests (Breiman output focus on cohe...
685,fit-a-fast-and-accurate-framework-for-solving,Autoencoder,NaN,NaN,NaN,NaN,NaN,could be\n𝑁 𝑀 NumberofMonteCarlosamples.\nobta...
686,improving-protein-gamma-turn-prediction-using,Capsule Network,NaN,NaN,NaN,NaN,NaN,were unsatisfactory with Matthew correlation c...


In [12]:
final_cleaned.to_csv("cleaned_data.csv", index=False)

In [13]:
final_cleaned

,paper_id,model1,model2,model3,model4,model5,model6,section
0,on-enhancing-speech-emotion-recognition-using,GAN,NaN,NaN,NaN,NaN,NaN,of a SVM trained on synthetic\nRealopenSMILE+i...
1,snap-ml-a-hierarchical-framework-for-machine,Logistic Regression,NaN,NaN,NaN,NaN,NaN,"InthisworkwehavedescribedSnapML,anewframeworkf..."
2,laplacian-smoothing-gradient-descent,Logistic Regression,NaN,NaN,NaN,NaN,NaN,on both a quadratic function and a\nsimple fin...
3,examining-the-use-of-neural-networks-for,SVM,NaN,NaN,NaN,NaN,NaN,show tures used to classify the image datasets...
4,genesis-of-basic-and-multi-layer-echo-state,Autoencoder,NaN,NaN,NaN,NaN,NaN,are. Feature extraction-based methods such as ...
...,...,...,...,...,...,...,...,...
683,a-generative-model-to-synthesize-eeg-data-for,GAN,SVM,DCGAN,NaN,NaN,NaN,show that the Epilepsy Society and Melbourne U...
684,using-machine-learning-to-calibrate-storm,Logistic Regression,NaN,NaN,NaN,NaN,NaN,. Random forests (Breiman output focus on cohe...
685,fit-a-fast-and-accurate-framework-for-solving,Autoencoder,NaN,NaN,NaN,NaN,NaN,could be\n𝑁 𝑀 NumberofMonteCarlosamples.\nobta...
686,improving-protein-gamma-turn-prediction-using,Capsule Network,NaN,NaN,NaN,NaN,NaN,were unsatisfactory with Matthew correlation c...
